## 11.2 准备文本数据

还是再重复一遍,现有的 nlp 模型只是在文本中提取模型,而不是真正意义上的理解.

如同前面 imdb 数据集的例子,模型无法处理输入的文本,需要将文本转换为数值张量.实现文本到张量的方法很多,但是一般都有相同的步骤.

- 首先是文本标准化,将其转化为小写字母,删除标点符号等.
- 将文本分割为单元(这个过程称之为标记),例如字符/词/词组等.
- 将每个这样的标记转成数字张量,通常进行这一步之前,需要将数据中所有标记进行索引.

![from_text_to_vectors](from_text_to_vectors.png)

## 文本标准化

考虑下面两个句子

- "sunset came. i was staring at the Mexico sky. Isnt nature splendid??"
- "Sunset came; I stared at the Mexico sky. Isn’t nature splendid?"

上面两个句子人读取之后会认为这是几乎一致的两个句子,但是转换成字符串比较看看?

- i 和 I
- Mexico 和 Mexico 都是墨西哥但是拼写不同
- Isnt 和 Isn’t
- stared 和 staring

机器学习无法知道 i 和 I 完全相同,就是大小写区别,é 是带重音的 e.这些规则和常识,机器无法使用,所以需要对文本进行标准化,尽力抹除这样的差异.



文本标准化也是特征工程的一种,旨在消除不希望模型处理的编码差异.这个过程并非机器学习独有,其他的工程例如搜索引擎也会进行.

最简单,最广泛的标准化方法: 转换成小写并删除标点符号.在样例上应用.

- "sunset came i was staring at the mexico sky isnt nature splendid"
- "sunset came i stared at the méxico sky isnt nature splendid"

比原来接近一些了,还有一种是特殊字符替换.将特殊字符替换为标准字符.这里的就是 e 替代 é,ae 替换 æ 等.

- "sunset came i was staring at the mexico sky isnt nature splendid"
- "sunset came i stared at the mexico sky isnt nature splendid"

抹除了一个单词的差异.

还有一种方法是非常高级的标准化,在机器学习背景的 nlp 中更少使用,将术语标准化.将一个术语(一个动词的多种形式/不同连接方式)转换成单一的表示.

- caught 或者 been catching 可以被替换为 catch.
- cats 可以替换为 cat.
- was staring 和 stared 可以被替换为 stare.

进行这样的替换,很可能两个意思相近的句子最终会是完全相同的编码.

- "sunset came i [stare] at the mexico sky isnt nature splendid"


文字标准化后,模型才可能在较少数据量情况下有更好的泛化能力.模型不需要理解 mexico 和 méxico 都是墨西哥,也不需要理会 Sunset 和 sunset 都是日落.当然文字标准化可能会抹除一些信息,(如果是在小说/文章中可能是一些非常精妙的文字描写--私注),所以进行文本标准化要始终看上下文的需要,加入是从采访文章中提取信息,那 `?` 肯定不能直接扔了吧? 必须单独做一个重点标记.\


## 文本拆分(文本标记化)


